# Use BERTweet Representations with LogisticRegression Softmax Classifier

In [1]:
from collections import Counter
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
# from transformers import AutoModel, AutoTokenizer 


import dataset
import vsm
import sst

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
TWITTER = 2
TWITTER_AIRLINES = 3
TWITTER_APPLE = 4

In [3]:
twitter_train, twitter_validate, twitter_test =  dataset.dataset_reader(TWITTER_APPLE)
[twitter_train, twitter_validate, twitter_test] = list(map(lambda ds : dataset.prune_columns(TWITTER_APPLE, ds), [twitter_train, twitter_validate, twitter_test]))

In [4]:
# bert_weights_name = 'bert-base-cased'
# bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
# bert_model = BertModel.from_pretrained(bert_weights_name)
# model = BertForSequenceClassification.from_pretrained(bert_weights_name)
# Unique values of sentiment
twitter_sentiment_labels = twitter_train['sentiment'].unique()

In [5]:
twitter_train.size, twitter_validate.size, twitter_test.size

(12436, 1552, 1556)

In [6]:
%%time
bertweet_experiment1500 = sst.experiment(
    twitter_train[:1500], # 
    dataset.bert_tweet_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[twitter_validate[:1000]],
    vectorize=False)

              precision    recall  f1-score   support

           1      0.689     0.658     0.673       111
           3      0.761     0.904     0.826       229
           5      0.700     0.171     0.275        41
not_relevant      0.000     0.000     0.000         7

    accuracy                          0.740       388
   macro avg      0.537     0.433     0.443       388
weighted avg      0.720     0.740     0.709       388

CPU times: user 39min 6s, sys: 2.28 s, total: 39min 8s
Wall time: 9min 53s


/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [7]:
%%time
bertweet_experiment_full = sst.experiment(
    twitter_train, # 
    dataset.bert_tweet_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[twitter_validate],
    vectorize=False)

              precision    recall  f1-score   support

           1      0.747     0.667     0.705       111
           3      0.772     0.917     0.838       229
           5      0.824     0.341     0.483        41
not_relevant      0.000     0.000     0.000         7

    accuracy                          0.768       388
   macro avg      0.586     0.481     0.506       388
weighted avg      0.757     0.768     0.747       388

CPU times: user 1h 10min 8s, sys: 6.1 s, total: 1h 10min 14s
Wall time: 17min 45s


/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [8]:
bertweet_experiment_full.keys()

dict_keys(['model', 'phi', 'train_dataset', 'assess_datasets', 'predictions', 'metric', 'scores'])

In [9]:
bertweet_experiment_full['scores']

[0.5064609696862433]

In [10]:
bertweet_experiment_full['metric']

'safe_macro_f1'

In [11]:
bertweet_experiment_full['model']

LogisticRegression(multi_class='ovr', solver='liblinear')

----

# Test BERTweet trained on Tweets on test set

In [12]:
import importlib
importlib.reload(sst)

<module 'sst' from '/home/vicaran93/repos/CS224-final-project/sst.py'>

In [13]:
%%time
bertweet_test = sst.evaluate(
    bertweet_experiment_full['model'],
    bertweet_experiment_full['phi'],
    assess_dataframes=[twitter_test],
    vectorizer=bertweet_experiment_full['assess_datasets'][0]['vectorizer'],
    vectorize=False
)

              precision    recall  f1-score   support

           1      0.781     0.685     0.730       130
           3      0.729     0.901     0.806       212
           5      0.462     0.171     0.250        35
not_relevant      0.000     0.000     0.000        12

    accuracy                          0.735       389
   macro avg      0.493     0.439     0.446       389
weighted avg      0.700     0.735     0.705       389

CPU times: user 7min 50s, sys: 856 ms, total: 7min 50s
Wall time: 1min 58s


/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/vicaran93/anaconda3/envs/nlu/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [16]:
type(bertweet_test['predictions'][0])

numpy.ndarray

In [14]:
predictions_fname ='results/BERTweet_predictions_on_twitter_test_apple.csv'
df = bertweet_test['predictions'][0]
pd.DataFrame(df).to_csv(predictions_fname)

In [15]:
encoding_fname ='results/BERTweet_encodings_on_twitter_test_apple.csv'
encoded_test = bertweet_test['assess_datasets'][0]
pd.DataFrame(encoded_test).to_csv(encoding_fname)

In [16]:
predictions_df = pd.DataFrame(df)
predictions_df = predictions_df.set_index(twitter_test.index)
predictions_df

,0
12,3
15,3
38,3
43,3
45,3
...,...
3853,1
3869,3
3870,3
3874,3


In [17]:
twitter_test['BERTweet_sentiment'] = predictions_df

In [18]:
twitter_test

,dataset,tweet_id,text,sentiment,BERTweet_sentiment
12,twitter_apple,623495525,"The Best-Designed #iPhone #Apps In the World, ...",3,3
15,twitter_apple,623495528,#aapl @applenws Thanks to the non factual dumb...,3,3
38,twitter_apple,623495551,RT @thehill: Justice Department cites 18th cen...,3,3
43,twitter_apple,623495556,RT @thehill: Justice Department cites 18th cen...,1,3
45,twitter_apple,623495558,RT @thehill: Justice Department cites 18th cen...,3,3
...,...,...,...,...,...
3853,twitter_apple,623499383,So @Apple just gave me a huge middle finger to...,1,1
3869,twitter_apple,623499399,Apple Is Warming Up To Social Media: Apple is ...,3,3
3870,twitter_apple,623499400,Apple Is Warming Up To Social Media: Apple is ...,3,3
3874,twitter_apple,623499404,Apple Is Warming Up To Social Media: Apple is ...,3,3


In [19]:
test_predictions_fname ='results/BERTweet_predictions_added_to_twitter_test_apple.csv'
twitter_test.to_csv(test_predictions_fname)

In [20]:
correct = twitter_test[twitter_test['sentiment'] == twitter_test['BERTweet_sentiment']]

In [21]:
correct

,dataset,tweet_id,text,sentiment,BERTweet_sentiment
12,twitter_apple,623495525,"The Best-Designed #iPhone #Apps In the World, ...",3,3
15,twitter_apple,623495528,#aapl @applenws Thanks to the non factual dumb...,3,3
38,twitter_apple,623495551,RT @thehill: Justice Department cites 18th cen...,3,3
45,twitter_apple,623495558,RT @thehill: Justice Department cites 18th cen...,3,3
46,twitter_apple,623495559,@thehill @Apple I cite the us constitution whe...,1,1
...,...,...,...,...,...
3853,twitter_apple,623499383,So @Apple just gave me a huge middle finger to...,1,1
3869,twitter_apple,623499399,Apple Is Warming Up To Social Media: Apple is ...,3,3
3870,twitter_apple,623499400,Apple Is Warming Up To Social Media: Apple is ...,3,3
3874,twitter_apple,623499404,Apple Is Warming Up To Social Media: Apple is ...,3,3


In [22]:
incorrect = twitter_test[twitter_test['sentiment'] != twitter_test['BERTweet_sentiment']]

In [23]:
incorrect

,dataset,tweet_id,text,sentiment,BERTweet_sentiment
43,twitter_apple,623495556,RT @thehill: Justice Department cites 18th cen...,1,3
273,twitter_apple,623495786,#AAPL $AAPL daily....wedge... http://t.co/RSkk...,not_relevant,3
311,twitter_apple,623495824,$AAPL #AAPL green right now. Hope it stays.,3,5
337,twitter_apple,623495850,TheStreet: #Android Phones to Dominate Throug...,5,3
351,twitter_apple,623495864,CNBCTV: #Cramer: Trading #Apple suckers game ...,1,3
...,...,...,...,...,...
3508,twitter_apple,623499038,I applied for @apple's 'head of social media' ...,3,1
3567,twitter_apple,623499097,@LydiasLyfe @BionicBombshell @Apple Apple bett...,3,1
3728,twitter_apple,623499258,Apple again orders app developer to remove a u...,3,1
3742,twitter_apple,623499272,"Apple Inc., GoPro Inc Products Are Go-To Gifts...",5,3


# Save Model

In [24]:
import pickle
model_fname = 'models/BERTweet_twitter_apple.sav'
pickle.dump(bertweet_experiment_full, open(model_fname, 'wb'))